In [57]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import matplotlib.gridspec as gridspec
import ast
import sys
sys.path.append('machine-scientist/')
sys.path.append('machine-scientist/Prior/')
from mcmc import *
from parallel import *
from fit_prior import read_prior_par
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error

In [51]:
def clean_index(dataframe):
    dataframe.set_index('Unnamed: 0', inplace=True)
    dataframe.index.name = None
    dataframe= dataframe.reset_index(drop=True)
    return dataframe

def add_bms_pred(dataframe, bms_trace, number_param):
    VARS = ['x1',]
    x = dn[[c for c in VARS]].copy()
    y=dataframe.noise

    if number_param==10:
        prior_par = read_prior_par('machine-scientist/Prior/final_prior_param_sq.named_equations.nv1.np10.2017-10-18 18:07:35.089658.dat')
    elif number_param==20:
        prior_par = read_prior_par('machine-scientist/Prior/final_prior_param_sq.named_equations.nv1.np20.maxs200.2024-05-10 162907.551306.dat')

    #mdl model
    minrow = bms_trace[bms_trace.H == min(bms_trace.H)].iloc[0]
    minH, minexpr, minparvals = minrow.H, minrow.expr, ast.literal_eval(minrow.parvals)

    t = Tree(
        variables=list(x.columns),
        parameters=['a%d' % i for i in range(number_param)],
        x=x, y=y,
        prior_par=prior_par,
        max_size=200,
        from_string=minexpr,
    )

    t.set_par_values(deepcopy(minparvals))

    dplot = deepcopy(dn)
    dplot['ybms'] = t.predict(x)

    return dplot
    
    

In [52]:
#Read NN and BMS data
function='tanh' #tanh, leaky_ReLU
realizations=2
N=9
#sigmas=[sigma_y for sigma_y in np.arange(0,0.2,0.02)]
sigmas=[0.02, 0.04,0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20]


runid=0
NPAR=10 #10, 20
steps=50000
train_size=60

rmse_nn_train=[];rmse_nn_test=[]
rmse_mdl_train=[];rmse_mdl_test=[]

mae_nn_train=[];mae_nn_test=[]
mae_mdl_train=[];mae_mdl_test=[]

n_index=[];r_index=[];sigma_index=[]

#Put mae and rmse of each simulation (on nn and bms) in a dataframe
for sigma in sigmas:
    for r in range(realizations+1):
        #Read NN data
        if sigma==0.1:
            file_model='NN_no_overfit_' + function + '_sigma_' + '0.10' + '_r_' + str(r) + '.csv'
        else:
            file_model='NN_no_overfit_' + function + '_sigma_' + str(sigma) + '_r_' + str(r) + '.csv'
            
        model_d='../data/trained_nns/' + file_model
        d=pd.read_csv(model_d)

        for n in range(N+1):
            n_index.append(n);r_index.append(r);sigma_index.append(sigma)
            
            dn=d[d['rep']==n]
            dn=clean_index(dn)

            #Read BMS data
            if sigma==0.1:
                filename='BMS_'+function+'_n_'+str(n)+'_sigma_'+'0.10'+ '_r_' + str(r) + '_trace_'+str(steps)+'_prior_'+str(NPAR)+ '.csv'
            else:
                filename='BMS_'+function+'_n_'+str(n)+'_sigma_'+str(sigma)+ '_r_' + str(r) + '_trace_'+str(steps)+'_prior_'+str(NPAR)+ '.csv'
        
            trace=pd.read_csv('../data/MSTraces/' + filename, sep=';', header=None, names=['t', 'H', 'expr', 'parvals', 'kk1', 'kk2','kk3'])
            dplot=add_bms_pred(dn, trace, NPAR)

            #Errors
            rmse_nn_train_i=root_mean_squared_error(dplot.loc[:train_size-1]['ymodel'],dplot.loc[:train_size -1]['y'])
            rmse_nn_train.append(rmse_nn_train_i)
            
            rmse_nn_test_i=root_mean_squared_error(dplot.loc[train_size-1:]['ymodel'],dplot.loc[train_size -1:]['y'])
            rmse_nn_test.append(rmse_nn_test_i)

            mae_nn_train_i=mean_absolute_error(dplot.loc[:train_size-1]['ymodel'],dplot.loc[:train_size -1]['y'])
            mae_nn_train.append(mae_nn_train_i)
            
            mae_nn_test_i=mean_absolute_error(dplot.loc[train_size-1:]['ymodel'],dplot.loc[train_size -1:]['y'])
            mae_nn_test.append(mae_nn_test_i)
    
            rmse_mdl_i=mean_squared_error(dplot.ybms,dn.y)
            
            rmse_mdl_train_i=root_mean_squared_error(dplot.loc[:train_size-1]['ybms'],dn.loc[:train_size-1]['y'])
            rmse_mdl_train.append(rmse_mdl_train_i)
            
            rmse_mdl_test_i=root_mean_squared_error(dplot.loc[train_size-1:]['ybms'],dn.loc[train_size-1:]['y'])
            rmse_mdl_test.append(rmse_mdl_test_i)

            mae_mdl_train_i=mean_absolute_error(dplot.loc[:train_size-1]['ybms'],dplot.loc[:train_size -1]['y'])
            mae_mdl_train.append(mae_mdl_train_i)
            
            mae_mdl_test_i=mean_absolute_error(dplot.loc[train_size-1:]['ybms'],dplot.loc[train_size -1:]['y'])
            mae_mdl_test.append(mae_mdl_test_i)

errors_df=pd.DataFrame({'sigma':sigma_index, 'mae_nn_train':mae_nn_train, 'mae_nn_test':mae_nn_test, 'mae_mdl_train':mae_mdl_train, 
                        'mae_mdl_test':mae_mdl_test, 'rmse_nn_train':rmse_nn_train, 'rmse_nn_test': rmse_nn_test, 
                        'rmse_mdl_train':rmse_mdl_train, 'rmse_mdl_test': rmse_mdl_test, 'n':n_index, 'r': r_index})
display(errors_df)

,sigma,mae_nn_train,mae_nn_test,mae_mdl_train,mae_mdl_test,rmse_nn_train,rmse_nn_test,rmse_mdl_train,rmse_mdl_test,n,r
0,0.02,0.038449,0.050661,0.010954,0.019700,0.047848,0.055290,0.012798,0.034677,0,0
1,0.02,0.012383,0.087417,0.016241,0.228908,0.015464,0.092709,0.020188,0.256246,1,0
2,0.02,0.048468,0.520923,0.008340,6.241106,0.113058,0.526282,0.011194,8.933877,2,0
3,0.02,0.022761,0.184491,0.006096,0.570502,0.048265,0.186005,0.008784,0.719521,3,0
4,0.02,0.009392,0.054881,0.003210,0.038630,0.011403,0.057578,0.005306,0.039809,4,0
...,...,...,...,...,...,...,...,...,...,...,...
295,0.20,0.059918,0.041152,0.025700,0.111698,0.077847,0.049771,0.039826,0.118180,5,2
296,0.20,0.138111,0.465183,0.047928,0.275630,0.168871,0.470017,0.074429,0.277328,6,2
297,0.20,0.091140,0.182207,0.095380,0.024869,0.117414,0.183915,0.110881,0.027818,7,2
298,0.20,0.111839,0.209496,0.070767,1.238419,0.135749,0.217007,0.076842,1.503048,8,2


In [53]:
test_df=errors_df[(errors_df['sigma']<=0.04) ]
#display(test_df)
#columns_errors=['mae_nn_train','mae_nn_test','mae_mdl_train','mae_mdl_test','rmse_nn_train','rmse_nn_test','rmse_mdl_train','rmse_mdl_test']
#Sum all values of r for each n, each sigma
#test_sum_df=test_df.groupby(['n', 'sigma'],as_index=False)[columns_errors].sum()
#display(test_sum_df)
#Get the mean of all values of r, each sigma
#test_mean_df=test_df.groupby(['n', 'sigma'],as_index=False)[columns_errors].mean()
#display(test_mean_df)

#total_test_df=errors_df

test_mean_errors_df=test_df.groupby(['n', 'sigma'],as_index=False)[columns_errors].mean()
total_means_errors_df=test_df.groupby(['sigma'],as_index=False)[columns_errors].mean() #la media total

#test_std_errors_df_total=test_df.groupby(['sigma'],as_index=False)[columns_errors].std() #la std total

test_std_errors_df=test_mean_errors_df.groupby(['sigma'],as_index=False)[columns_errors].std() #std sobre las medias
#display(test_std_errors_df)

display(test_df[test_df['sigma']==0.02])

display(total_means_errors_df[total_means_errors_df['sigma']==0.02])

display(test_mean_errors_df[test_mean_errors_df['sigma']==0.02])

display(test_std_errors_df[test_std_errors_df['sigma']==0.02])
#display(test_std_errors_df_total[test_std_errors_df_total['sigma']==0.02])

,sigma,mae_nn_train,mae_nn_test,mae_mdl_train,mae_mdl_test,rmse_nn_train,rmse_nn_test,rmse_mdl_train,rmse_mdl_test,n,r
0,0.02,0.038449,0.050661,0.010954,0.019700,0.047848,0.055290,0.012798,0.034677,0,0
1,0.02,0.012383,0.087417,0.016241,0.228908,0.015464,0.092709,0.020188,0.256246,1,0
2,0.02,0.048468,0.520923,0.008340,6.241106,0.113058,0.526282,0.011194,8.933877,2,0
3,0.02,0.022761,0.184491,0.006096,0.570502,0.048265,0.186005,0.008784,0.719521,3,0
4,0.02,0.009392,0.054881,0.003210,0.038630,0.011403,0.057578,0.005306,0.039809,4,0
5,0.02,0.011894,0.071605,0.006698,0.097954,0.015032,0.089426,0.008549,0.123879,5,0
6,0.02,0.006727,0.071026,0.008436,0.031277,0.008800,0.074453,0.011928,0.033395,6,0
7,0.02,0.008590,0.029799,0.007644,178.264541,0.011636,0.031023,0.011447,519.386938,7,0
8,0.02,0.024349,0.266486,0.010406,0.279775,0.049119,0.269066,0.013890,0.325837,8,0
9,0.02,0.032108,0.079216,0.006219,0.045368,0.040759,0.087545,0.008317,0.055860,9,0


,sigma,mae_nn_train,mae_nn_test,mae_mdl_train,mae_mdl_test,rmse_nn_train,rmse_nn_test,rmse_mdl_train,rmse_mdl_test
0,0.02,0.024488,0.126648,0.008538,12.255278,0.04064,0.131665,0.011169,35.119318


,n,sigma,mae_nn_train,mae_nn_test,mae_mdl_train,mae_mdl_test,rmse_nn_train,rmse_nn_test,rmse_mdl_train,rmse_mdl_test
0,0,0.02,0.038290,0.042746,0.010459,0.049737,0.047936,0.047007,0.012149,0.069522
2,1,0.02,0.013289,0.112173,0.016125,0.234434,0.017184,0.120072,0.019763,0.261306
4,2,0.02,0.048812,0.532540,0.009131,2.257830,0.115570,0.537947,0.012154,3.183713
6,3,0.02,0.022300,0.165098,0.006206,0.604693,0.046708,0.168113,0.008849,0.763309
8,4,0.02,0.036643,0.037187,0.003375,0.038199,0.059056,0.039048,0.005432,0.039392
10,5,0.02,0.034836,0.066184,0.006698,0.097954,0.044079,0.081194,0.008549,0.123879
12,6,0.02,0.007322,0.059330,0.008193,0.029449,0.009609,0.062876,0.010749,0.031278
14,7,0.02,0.010468,0.058638,0.008265,118.857961,0.016014,0.059448,0.011573,346.276967
16,8,0.02,0.015037,0.123393,0.010211,0.322044,0.024994,0.126618,0.013717,0.372963
18,9,0.02,0.017886,0.069191,0.006718,0.060478,0.025252,0.074324,0.008757,0.070851


,sigma,mae_nn_train,mae_nn_test,mae_mdl_train,mae_mdl_test,rmse_nn_train,rmse_nn_test,rmse_mdl_train,rmse_mdl_test
0,0.02,0.014108,0.148175,0.003389,37.46248,0.031014,0.148354,0.003853,109.333855


In [56]:
columns_errors=['mae_nn_train','mae_nn_test','mae_mdl_train','mae_mdl_test','rmse_nn_train','rmse_nn_test','rmse_mdl_train','rmse_mdl_test']

#Mean values of 30 (3*10) simulations
mean_errors_df=errors_df.groupby(['sigma'],as_index=False)[columns_errors].mean()
display(mean_errors_df)


#Error
#1. For every sigma and every n, calculate means over the realizations
mean_n_over_r_df=errors_df.groupby(['n', 'sigma'],as_index=False)[columns_errors].mean()
display(mean_n_over_r_df)

#2. Calculate stds over means of over realizations
std_n_over_r_df=mean_n_over_r_df.groupby(['sigma'],as_index=False)[columns_errors].std() 


#3. sem: divide by sqare root of realizations
sem_n_over_r_df=std_n_over_r_df[columns_errors]/sqrt(realizations+1)



#rename columns
std_n_over_r_df=std_errors_df.rename(columns={'mae_nn_train':  'mae_nn_train_std',  'mae_nn_test':  'mae_nn_test_std', 
                                            'mae_mdl_train': 'mae_mdl_train_std', 'mae_mdl_test': 'mae_mdl_test_std',
                                            'rmse_nn_train': 'rmse_nn_train_std', 'rmse_nn_test': 'rmse_nn_test_std' , 
                                            'rmse_mdl_train':'rmse_mdl_train_std','rmse_mdl_test':'rmse_mdl_test_std'})


sem_n_over_r_df=sem_n_over_r_df.rename(columns={'mae_nn_train':  'sem_mae_nn_train',  'mae_nn_test':  'sem_mae_nn_test', 
                                            'mae_mdl_train': 'sem_mae_mdl_train', 'sem_mae_mdl_test': 'sem_mae_mdl_test',
                                            'rmse_nn_train': 'sem_rmse_nn_train', 'sem_rmse_nn_test': 'sem_rmse_nn_test' , 
                                            'rmse_mdl_train':'sem_rmse_mdl_train','sem_rmse_mdl_test':'sem_rmse_mdl_test'})


errors_statistics_new=mean_n_over_r_df.join(std_n_over_r_df.set_index('sigma'), on='sigma')
display(errors_statistics_new)
errors_statistics_new=errors_statistics_new.join(sem_n_over_r_df.set_index('sigma'), on='sigma')



#------------------------------------------------------------------------------------------------------------------------------------------
#Calculate stds over sigma and rename columns
#std_errors_df=errors_df.groupby(['sigma'],as_index=False)[columns_errors].std()

#std_errors_df=std_errors_df.rename(columns={'mae_nn_train':  'mae_nn_train_std',  'mae_nn_test':  'mae_nn_test_std', 
                                           # 'mae_mdl_train': 'mae_mdl_train_std', 'mae_mdl_test': 'mae_mdl_test_std',
                                           #'rmse_nn_train': 'rmse_nn_train_std', 'rmse_nn_test': 'rmse_nn_test_std' , 
                                            #'rmse_mdl_train':'rmse_mdl_train_std','rmse_mdl_test':'rmse_mdl_test_std'})

#Calculate sdm
#std_errors_df[['mae_nn_train_sdm', 'mae_nn_test_sdm', 'mae_mdl_train_sdm', 'mae_mdl_test_sdm',
#               'rmse_nn_train_sdm', 'rmse_nn_test_sdm', 'rmse_mdl_train_sdm', 'rmse_mdl_test_sdm']]=\
#std_errors_df[['mae_nn_train_std', 'mae_nn_test_std', 'mae_mdl_train_std', 'mae_mdl_test_std',
#               'rmse_nn_train_std', 'rmse_nn_test_std', 'rmse_mdl_train_std', 'rmse_mdl_test_std']]/np.sqrt(N*(realizations+1))
#display(std_errors_df)


#join means stds and sdms
#errors_statistics_df=mean_errors_df.join(std_errors_df.set_index('sigma'), on='sigma')

#display(errors_statistics_df)

#save error dataframes
#errors_statistics_df.to_csv('../data/'+ 'errors_statistics' + str(function) + '.csv')

,sigma,mae_nn_train,mae_nn_test,mae_mdl_train,mae_mdl_test,rmse_nn_train,rmse_nn_test,rmse_mdl_train,rmse_mdl_test
0,0.02,0.024488,0.126648,0.008538,1.225528e+01,0.040640,0.131665,0.011169,3.511932e+01
1,0.04,0.023552,0.106514,0.018319,7.384092e-02,0.032968,0.113084,0.022554,8.726131e-02
2,0.06,0.030517,0.196093,0.028401,1.818815e-01,0.043695,0.208920,0.035352,2.146793e-01
3,0.08,0.040236,0.118984,0.037672,2.752581e-01,0.054732,0.124087,0.046854,3.346801e-01
4,0.10,0.046230,0.224757,0.040629,1.984707e-01,0.067793,0.231479,0.052115,2.410020e-01
5,0.12,0.054552,0.200388,0.049357,5.161457e-01,0.076709,0.205137,0.061820,6.900875e-01
6,0.14,0.065086,0.144585,0.058810,3.253921e-01,0.089562,0.151999,0.076177,3.808002e-01
7,0.16,0.067557,0.297781,0.056738,1.959929e-01,0.093146,0.310959,0.068208,2.083971e-01
8,0.18,0.095329,0.320198,0.086780,3.808135e-01,0.129272,0.338743,0.108729,4.228715e-01
9,0.20,0.100726,0.422235,0.069977,3.570581e+30,0.141464,0.434688,0.093550,1.636161e+31


,n,sigma,mae_nn_train,mae_nn_test,mae_mdl_train,mae_mdl_test,rmse_nn_train,rmse_nn_test,rmse_mdl_train,rmse_mdl_test
0,0,0.02,0.038290,0.042746,0.010459,0.049737,0.047936,0.047007,0.012149,0.069522
1,0,0.04,0.038073,0.015235,0.036072,0.011795,0.044877,0.019214,0.043559,0.013066
2,0,0.06,0.032296,0.054426,0.042288,0.023119,0.040256,0.058724,0.051611,0.029146
3,0,0.08,0.046319,0.083422,0.036188,0.324084,0.061758,0.087729,0.044893,0.391931
4,0,0.10,0.042544,0.053966,0.042326,0.013374,0.054433,0.060518,0.055185,0.020614
...,...,...,...,...,...,...,...,...,...,...
95,9,0.12,0.067373,0.250060,0.072934,0.851467,0.087076,0.255091,0.081959,0.983302
96,9,0.14,0.088436,0.273661,0.065584,0.347172,0.133017,0.294882,0.079152,0.386440
97,9,0.16,0.124249,0.722254,0.043076,0.043316,0.186357,0.734369,0.053509,0.043721
98,9,0.18,0.089819,0.075660,0.074846,1.004900,0.122399,0.088637,0.086087,1.139778


,n,sigma,mae_nn_train,mae_nn_test,mae_mdl_train,mae_mdl_test,rmse_nn_train,rmse_nn_test,rmse_mdl_train,rmse_mdl_test,...,rmse_mdl_train_std,rmse_mdl_test_std,mae_nn_train_sdm,mae_nn_test_sdm,mae_mdl_train_sdm,mae_mdl_test_sdm,rmse_nn_train_sdm,rmse_nn_test_sdm,rmse_mdl_train_sdm,rmse_mdl_test_sdm
0,0,0.02,0.038290,0.042746,0.010459,0.049737,0.047936,0.047007,0.012149,0.069522,...,0.003824,1.316494e+02,0.004232,0.028671,0.000646,8.687306e+00,0.007786,0.028709,0.000736,2.533594e+01
1,0,0.04,0.038073,0.015235,0.036072,0.011795,0.044877,0.019214,0.043559,0.013066,...,0.010576,8.772654e-02,0.002157,0.018359,0.001693,1.417583e-02,0.003316,0.018940,0.002035,1.688298e-02
2,0,0.06,0.032296,0.054426,0.042288,0.023119,0.040256,0.058724,0.051611,0.029146,...,0.015893,3.238518e-01,0.002786,0.033409,0.002452,5.043231e-02,0.005094,0.034043,0.003059,6.232530e-02
3,0,0.08,0.046319,0.083422,0.036188,0.324084,0.061758,0.087729,0.044893,0.391931,...,0.026520,4.572164e-01,0.002158,0.021433,0.004374,7.043805e-02,0.003484,0.021354,0.005104,8.799135e-02
4,0,0.10,0.042544,0.053966,0.042326,0.013374,0.054433,0.060518,0.055185,0.020614,...,0.021292,3.914075e-01,0.003650,0.043948,0.003230,5.986438e-02,0.008087,0.044072,0.004098,7.532641e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9,0.12,0.067373,0.250060,0.072934,0.851467,0.087076,0.255091,0.081959,0.983302,...,0.022740,1.710149e+00,0.003718,0.032561,0.004100,2.066483e-01,0.005863,0.032366,0.004376,3.291184e-01
96,9,0.14,0.088436,0.273661,0.065584,0.347172,0.133017,0.294882,0.079152,0.386440,...,0.036537,5.223886e-01,0.004458,0.026943,0.006084,8.074543e-02,0.007175,0.027715,0.007032,1.005337e-01
97,9,0.16,0.124249,0.722254,0.043076,0.043316,0.186357,0.734369,0.053509,0.043721,...,0.042382,1.354749e-01,0.006665,0.051405,0.007022,2.527719e-02,0.010409,0.051514,0.008156,2.607215e-02
98,9,0.18,0.089819,0.075660,0.074846,1.004900,0.122399,0.088637,0.086087,1.139778,...,0.036479,3.158029e-01,0.005491,0.083168,0.006486,5.414365e-02,0.008524,0.087833,0.007020,6.077630e-02


KeyError: "None of ['sigma'] are in the columns"

In [5]:
#Plot rmse
Extensions=['.png', '.pdf']

#Fonts and sizes                                                                                    
size_axis=7;size_ticks=6;size_title=5
line_w=1;marker_s=3 #width and marker size                                                          
m_size=6

#Define figure size in cm                                                                           
cm = 1/2.54 #convert inch to cm                                                                     
width = 8*cm; height=6*cm
rows=3;cols=10

width_panel = width*cols
height_panel= height*rows

fig=figure(figsize=(width_panel,height_panel), dpi=300)  
gs=gridspec.GridSpec(rows,cols)                                                                                                                         
gs.update(left=0.1,right=0.98,bottom=0.15,top=0.90,wspace=0.35,hspace=0.1) 

for row in range(len(rows)):
    print(row)
    for col in range(len(cols)): 
        print(col)

        ax_ij=plt.subplot(gs[i,j])
        plt.plot(dn['x1'], dn['ymodel'],linewidth=line_w, color='red', label='ann.  MSE_train= %.2E, MSE_test= %.2E' % ( MAE_nn_train, MAE_nn_test))

SyntaxError: incomplete input (2887671460.py, line 23)